In [ ]:
import numpy as np

# linear congruential generator
def lcg(modulus, a, c, X):
    while True:
        X = (a * X + c) % modulus
        yield X

def generate_normal_samples(N):
    # Parameters for the Linear Congruential Generator (LCG)
    a = 39373
    c = 0
    k = 2**31 - 1
    X0 = 1
    generator = lcg(k, a, c, X0)
    
    # Box-Muller Method
    def box_muller():
        while True:
            u1 = next(generator) / k
            u2 = next(generator) / k
            v1 = 2 * u1 - 1
            v2 = 2 * u2 - 1
            s = v1**2 + v2**2
            if s <= 1:
                break
        factor = np.sqrt(-2 * np.log(s) / s)
        z1 = v1 * factor
        z2 = v2 * factor
        return z1, z2

    # Generate N normal distribution samples using Box-Muller Method
    normal_samples = []
    for _ in range(N//2):
        z1, z2 = box_muller()
        normal_samples.append(z1)
        normal_samples.append(z2)
    return normal_samples


normal_samples = generate_normal_samples(10000*2**9+1)

In [2]:
from scipy.stats import norm
import numpy as np

# Given parameters
S0 = 42  # spot price
K = 40  # strike price
T = 7/12  # time to maturity
r = 0.04  # risk-free rate
q = 0.015  # dividend yield
sigma = 0.28  # volatility
B = 36  # barrier

def black_scholes_call(S, K, T, r, q, sigma):
    d1 = (np.log(S / K) + (r - q + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    term1 = S * np.exp(-q * T) * norm.cdf(d1)
    term2 = K * np.exp(-r * T) * norm.cdf(d2)
    return term1 - term2


# Function to calculate the exact value of a down-and-out call option
def down_and_out_call(S, K, B, T, r, q, sigma):
    a = (r - q - sigma**2 / 2) / sigma**2
    term1 = black_scholes_call(S, K, T, r, q, sigma)
    term2 = (B/S)**(2 * a) * black_scholes_call(B**2 / S, K, T, r, q, sigma)
    return term1 - term2

# Calculate the exact value of the down-and-out call option
exact_value = down_and_out_call(S0, K, B, T, r, q, sigma)
print(exact_value)


4.375599651961108


In [29]:
import pandas as pd
import math
# Function to simulate one path of the underlying asset price using the Geometric Brownian Motion
def simulate_path(S0, T, r, q, sigma, z, m, B, K):
    dt = T / m
    S = S0
    for j in range(1, m + 1):
        S = S * np.exp((r - q - sigma**2 / 2) * dt + sigma * np.sqrt(dt) * z[j - 1])
        if S <= B:
            return 0
    return max(S - K, 0)


# Function to approximate the value of a down-and-out call option using Monte Carlo simulation
def monte_carlo_dao(S0, K, B, T, r, q, sigma, z_samples, n, m):
    V = []
    for i in range(n):
        z = z_samples[i * m:(i + 1) * m]
        payoff = simulate_path(S0, T, r, q, sigma, z, m, B, K)
        V.append(payoff)
    V = np.exp(-r * T) * np.array(V)  # Discount the payoffs back to present value
    V_mean = np.mean(V)
    return V_mean, V

# Parameters for the Monte Carlo simulation
m = 200  # number of time intervals
n = 50  # number of paths

res = pd.DataFrame(columns=['N', 'V_n', '|Cdao-V_n|', 'm_k', 'n_k', 'V_nk', '|Cdao-V_nk|'])

normal_samples = np.array(normal_samples)
for k in range(10):
    V_n, V1 = monte_carlo_dao(S0, K, B, T, r, q, sigma, normal_samples, n, m)
    N_k = n * m
    m_k = math.ceil(N_k**(1/3)*T**(2/3))
    n_k = math.floor(N_k/m_k)
    V_nk, V2 = monte_carlo_dao(S0, K, B, T, r, q, sigma, normal_samples, n_k, m_k)
    
    print('N_k={}, V_mean={}, |Cdao-V_mean|={}'.format(N_k, V_n, abs(exact_value - V_n)))
    res.loc[N_k] = [n, V_n, abs(exact_value - V_n), m_k, n_k, V_nk, abs(exact_value - V_nk)]
    n *= 2
res

N_k=10000, V_mean=4.280800885038715, |Cdao-V_mean|=0.0947987669223922
N_k=20000, V_mean=4.586578358287422, |Cdao-V_mean|=0.21097870632631466
N_k=40000, V_mean=4.3980507006743785, |Cdao-V_mean|=0.02245104871327097
N_k=80000, V_mean=4.347925586634464, |Cdao-V_mean|=0.02767406532664385
N_k=160000, V_mean=4.250593041366492, |Cdao-V_mean|=0.1250066105946157
N_k=320000, V_mean=4.360514679741517, |Cdao-V_mean|=0.01508497221959093
N_k=640000, V_mean=4.480754343491606, |Cdao-V_mean|=0.10515469153049839
N_k=1280000, V_mean=4.43997228014777, |Cdao-V_mean|=0.06437262818666234
N_k=2560000, V_mean=4.380950948459192, |Cdao-V_mean|=0.0053512964980848565
N_k=5120000, V_mean=4.423512884711982, |Cdao-V_mean|=0.047913232750874


,N,V_n,|Cdao-V_n|,m_k,n_k,V_nk,|Cdao-V_nk|
10000,50.0,4.280801,0.094799,16.0,625.0,4.408881,0.033282
20000,100.0,4.586578,0.210979,19.0,1052.0,4.625003,0.249403
40000,200.0,4.398051,0.022451,24.0,1666.0,4.666725,0.291125
80000,400.0,4.347926,0.027674,31.0,2580.0,4.577915,0.202315
160000,800.0,4.250593,0.125007,38.0,4210.0,4.548373,0.172773
320000,1600.0,4.360515,0.015085,48.0,6666.0,4.560420,0.184820
640000,3200.0,4.480754,0.105155,61.0,10491.0,4.559870,0.184270
1280000,6400.0,4.439972,0.064373,76.0,16842.0,4.524505,0.148906
2560000,12800.0,4.380951,0.005351,96.0,26666.0,4.488441,0.112842
5120000,25600.0,4.423513,0.047913,121.0,42314.0,4.450953,0.075353


In [30]:
res.to_csv('q2-res.csv', index=True)